<a href="https://colab.research.google.com/github/Camilo-a17/Analisis-/blob/master/Analisis%20Tesla_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from scipy import stats

df = pd.read_csv("tesla_data.csv")

In [9]:
print(df.columns)
df.describe

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Cumulative Open', 'Price Change'],
      dtype='object')


<bound method NDFrame.describe of            Date        Open        High         Low       Close   Adj Close  \
0    2022-08-29  282.829987  287.739990  280.700012  284.820007  284.820007   
1    2022-08-30  287.869995  288.480011  272.649994  277.700012  277.700012   
2    2022-08-31  280.619995  281.250000  271.809998  275.609985  275.609985   
3    2022-09-01  272.579987  277.579987  266.149994  277.160004  277.160004   
4    2022-09-02  281.070007  282.350006  269.079987  270.209991  270.209991   
..          ...         ...         ...         ...         ...         ...   
246  2023-08-22  240.250000  240.820007  229.550003  233.190002  233.190002   
247  2023-08-23  229.339996  238.979996  229.289993  236.860001  236.860001   
248  2023-08-24  238.660004  238.919998  228.179993  230.039993  230.039993   
249  2023-08-25  231.309998  239.000000  230.350006  238.589996  238.589996   
250  2023-08-28  242.580002  244.380005  235.360001  238.820007  238.820007   

        Volume  C

In [10]:
# Supongamos que ya tienes un DataFrame de pandas llamado 'df' con la columna 'Close' y 'Date'.

# Calcular los promedios móviles simples
df['SMA_50'] = df['Close'].rolling(window=50).mean()
df['SMA_200'] = df['Close'].rolling(window=200).mean()

# Crear una figura de Plotly
fig = go.Figure()

# Agregar la serie temporal del precio de cierre
fig.add_trace(go.Scatter(x=df['Date'], y=df['Close'], mode='lines', name='Precio de Cierre de Tesla'))

# Agregar los promedios móviles simples
fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA_50'], mode='lines', name='SMA 50 días'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA_200'], mode='lines', name='SMA 200 días'))

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Serie Temporal del Precio de Cierre de Tesla con Plotly',
    xaxis=dict(title='Fecha'),
    yaxis=dict(title='Precio de Cierre'),
    legend=dict(x=0.7, y=0.95)
)

# Mostrar el gráfico interactivo
fig.show()



Análisis de Volatilidad:

In [11]:
# Calcular los rendimientos diarios
df['Daily Returns'] = df['Close'].pct_change()

# Detectar valores atípicos usando z-score
z_scores = np.abs(stats.zscore(df['Daily Returns'].dropna()))
threshold = 3  # Umbral de valor atípico
outliers = np.where(z_scores > threshold)

# Crear el gráfico con los puntos atípicos marcados
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Daily Returns'], mode='lines', name='Rendimientos Diarios'))
fig.add_trace(go.Scatter(x=df['Date'].iloc[outliers], y=df['Daily Returns'].iloc[outliers], mode='markers', name='Puntos Anormales', marker=dict(color='red', size=8)))
fig.update_layout(title='Volatilidad de los Rendimientos Diarios de Tesla con Puntos Anormales')
fig.show()



Análisis de Rendimiento:

In [23]:
# Calcular el rendimiento acumulado
df['Cumulative Returns'] = (1 + df['Daily Returns']).cumprod() - 1

# Detectar valores atípicos usando z-score
z_scores = (df['Cumulative Returns'] - df['Cumulative Returns'].mean()) / df['Cumulative Returns'].std()
threshold = 3  # Umbral de valor atípico
outliers = z_scores[abs(z_scores) > threshold]

# Crear el gráfico con los puntos atípicos marcados
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Cumulative Returns'], mode='lines', name='Rendimiento Acumulado'))
fig.add_trace(go.Scatter(x=df['Date'].iloc[outliers.index], y=df['Cumulative Returns'].iloc[outliers.index], mode='markers', name='Puntos Anormales', marker=dict(color='red', size=8)))
fig.update_layout(title='Rendimiento Acumulado de Tesla')
fig.show()



Análisis de Volumen:



In [13]:
# Detectar valores atípicos en el volumen de operaciones usando z-score
z_scores = np.abs(stats.zscore(df['Volume']))
threshold = 3  # Umbral de valor atípico
outliers = np.where(z_scores > threshold)

# Crear el gráfico con los puntos atípicos marcados
fig = go.Figure()
fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], name='Volumen de Operaciones', marker_color='blue'))
fig.add_trace(go.Scatter(x=df['Date'].iloc[outliers], y=df['Volume'].iloc[outliers], mode='markers', name='Puntos Anormales', marker=dict(color='red', size=8)))
fig.update_layout(title='Volumen de Operaciones de Tesla con Puntos Anormales')
fig.show()


Análisis de Cambio de Precio con Puntos Anormales:

In [22]:
# Detectar valores atípicos en los cambios de precio usando z-score
z_scores = np.abs(stats.zscore(df['Price Change']))
threshold = 3  # Umbral de valor atípico
outliers = np.where(z_scores > threshold)

# Crear el gráfico con los puntos atípicos marcados
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price Change'], mode='lines', name='Cambios de Precio'))
fig.add_trace(go.Scatter(x=df['Date'].iloc[outliers], y=df['Price Change'].iloc[outliers], mode='markers', name='Puntos Anormales', marker=dict(color='red', size=8)))
fig.update_layout(title='Cambios de Precio de Tesla')
fig.show()


In [20]:
# Crear una figura de Waterfall Chart
fig = go.Figure(go.Waterfall(
    name="Precio de Apertura de TSLA",
    orientation="v",
    measure=["relative"] * len(df['Date']),  # Cada paso es relativo al anterior
    x=df['Date'],
    y=df['Open']
))

# Personalizar el diseño del gráfico
fig.update_layout(
    title='Waterfall Chart del Precio de Apertura de TSLA',
    xaxis=dict(title='Fecha'),
    yaxis=dict(title='Precio de Apertura'),
    showlegend=True
)

# Mostrar el gráfico interactivo
fig.show()

